

```
# This is formatted as code
```

# DMA Fall 21

**Note** : This entire lab will be manually evaluated.

Name : 'Yinglu Deng'

Collaborator : ''

# Lab 4: Neural Networks #


In [2]:
import pandas as pd
from sklearn.neural_network import MLPClassifier
from sklearn.svm import SVC

from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.preprocessing import LabelEncoder, OneHotEncoder
from sklearn.feature_extraction import DictVectorizer

from sklearn.pipeline import Pipeline
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV, ParameterGrid

import numpy as np

import warnings
warnings.filterwarnings("ignore")

In [3]:
!wget http://askoski.berkeley.edu/~zp/lab_4_training.csv
!wget http://askoski.berkeley.edu/~zp/lab_4_test.csv

df_train = pd.read_csv('./lab_4_training.csv')
df_test = pd.read_csv('./lab_4_test.csv')
df_train.head()

--2021-09-22 04:54:33--  http://askoski.berkeley.edu/~zp/lab_4_training.csv
Resolving askoski.berkeley.edu (askoski.berkeley.edu)... 169.229.192.179
Connecting to askoski.berkeley.edu (askoski.berkeley.edu)|169.229.192.179|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 79177 (77K) [text/csv]
Saving to: ‘lab_4_training.csv’

lab_4_training.csv  100%[===================>]  77.32K  57.7KB/s    in 1.3s    

2021-09-22 04:54:37 (57.7 KB/s) - ‘lab_4_training.csv’ saved [79177/79177]

--2021-09-22 04:54:37--  http://askoski.berkeley.edu/~zp/lab_4_test.csv
Resolving askoski.berkeley.edu (askoski.berkeley.edu)... 169.229.192.179
Connecting to askoski.berkeley.edu (askoski.berkeley.edu)|169.229.192.179|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 26519 (26K) [text/csv]
Saving to: ‘lab_4_test.csv’

lab_4_test.csv      100%[===================>]  25.90K  34.8KB/s    in 0.7s    

2021-09-22 04:54:38 (34.8 KB/s) - ‘lab_4_test.csv’ saved [26519/26

,Unnamed: 0,gender,age,year,eyecolor,height,miles,brothers,sisters,computertime,exercise,exercisehours,musiccds,playgames,watchtv
0,577,male,20,third,hazel,72.0,180.0,0,0,5.0,No,0.0,100.0,10.0,10.0
1,677,male,19,second,hazel,72.0,120.0,1,1,16.0,Yes,9.0,70.0,3.0,5.0
2,1738,male,20,second,brown,63.0,55.0,1,2,15.0,Yes,4.5,15.0,4.0,13.0
3,1355,male,20,third,green,78.0,200.0,0,0,10.0,Yes,9.0,20.0,10.0,10.0
4,891,female,19,second,green,67.0,280.0,2,0,4.0,Yes,2.0,164.0,0.0,2.0


***
### Question 1###
Calculate a baseline accuracy measure using the majority class, assuming a target variable of 'gender'. The majority class is the most common value of the target variable in a particular dataset. Accuracy is calculated as (true positives + true negatives) / (all negatives and positives)

**Question 1.a**  
Find the majority class in the training set. If you always predicted this class in the training set, what would your accuracy be?

In [4]:
# YOUR CODE HERE
train_group = df_train.groupby('gender').size()
train_group

gender
female    647
male      545
dtype: int64

In [5]:
label_list = list(train_group)
majority_train_accuracy = max(label_list) / np.sum(label_list)
majority_train_accuracy

0.5427852348993288

###ANSWER: The majority class is female. And the accuracy will be 0.5427852348993288.

**Question 1.b**   
If you always predicted this same class (majority from the training set) in the test set, what would your accuracy be?

In [6]:
# YOUR CODE HERE
test_group = df_test.groupby('gender').size()
test_group

gender
female    208
male      190
dtype: int64

In [7]:
label_list = list(test_group)
majority_test_accuracy = max(label_list) / np.sum(label_list)
majority_test_accuracy

0.5226130653266332

###ANSWER: The majority class is female. And the accuracy will be 0.5226130653266332.

***
### Question 2 ###
Get started with Neural Networks.

   
Choose a NN implementation (eg: scikit-learn) and specify which you choose. Be sure the implementation allows you to modify the number of hidden layers and hidden nodes per layer.  

NOTE: When possible, specify the logsig (sigmoid/logistc) function as the transfer function (another word for activation function) and use Levenberg-Marquardt backpropagation (lbfgs). It is possible to specify logistic in Sklearn MLPclassifier (Neural net).  

**Question 2.a**   
Train a neural network with a single 10 node hidden layer. Only use the Height feature of the dataset to predict the Gender. You will have to change Gender to a 0 and 1 class. After training, use your trained model to predict the class using the height feature from the training set. What was the accuracy of this prediction?

In [8]:
# YOUR CODE HERE
# use keras as a NN implementation
from keras.models import Sequential
from keras.layers import Dense
from sklearn import preprocessing

# use the height feature and transform the gender to binary
X_train = np.array(df_train[['height']])
df_train['gender_binary'] = df_train['gender'].apply(lambda x: 1 if x == 'female' else 0)
Y_train = np.array(df_train[['gender_binary']])

clf = MLPClassifier(hidden_layer_sizes=(10), activation = 'logistic',
                     solver='lbfgs', random_state=1)  #sgd = Stochastic Gradient descent. Also, check 'lbfgs’, ‘adam’}
                                                                #Default activation is 'relu'
                                                                #Default n_iter_no_change=10

clf.fit(X_train,Y_train)

print('Accuracy on training---')
y_pred_train=clf.predict(X_train)
print(accuracy_score(Y_train,y_pred_train))


Accuracy on training---
0.802013422818792


###ANSWER: The accuracy of this prediction for training set is 0.802013422818792.

**Question 2.b**  
Take the trained model from question 2.a and use it to predict the test set. This can be accomplished by taking the trained model and giving it the Height feature values from the test set. What is the accuracy of this model on the test set?

In [9]:
# YOUR CODE HERE
X_test = np.array(df_test[['height']])
df_test['gender_binary'] = df_test['gender'].apply(lambda x: 1 if x == 'female' else 0)
Y_test = np.array(df_test[['gender_binary']])

print('Accuracy on test---')
y_pred_test=clf.predict(X_test)
print(accuracy_score(Y_test,y_pred_test))

Accuracy on test---
0.7939698492462312


###ANSWER: The accuracy of this prediction for test set is 0.7939698492462312.

**Question 2.c**   
Neural Networks tend to prefer smaller, normalized feature values. Try taking the log of the height feature in both training and testing sets or use a Standard Scalar operation in SKlearn to centre and normalize the data between 0-1 for continuous values. Repeat question 2.a and 2.b with the log version or the normalized and centered version of this feature

In [10]:
# YOUR CODE HERE
from sklearn import preprocessing
# use a Standard Scalar operation in SKlearn to centre and normalize the data between 0-1 for continuous values
sc=preprocessing.StandardScaler()
X_train_scaled=sc.fit_transform(X_train)
X_test_scaled=sc.transform(X_test)

clf = MLPClassifier(hidden_layer_sizes=(10), activation = 'logistic',
                     solver='lbfgs', random_state=1)  #sgd = Stochastic Gradient descent. Also, check 'lbfgs’, ‘adam’}
                                                                #Default activation is 'relu'
                                                                #Default n_iter_no_change=10

clf.fit(X_train_scaled,Y_train)

print('Accuracy on training---')
y_pred_scaled_train=clf.predict(X_train_scaled)
print(accuracy_score(Y_train, y_pred_scaled_train))

print('')
print('Accuracy on test---')
y_pred_scaled_test=clf.predict(X_test_scaled)
print(accuracy_score(Y_test, y_pred_scaled_test))
 

Accuracy on training---
0.8439597315436241

Accuracy on test---
0.8542713567839196


###ANSWER: After normalizing the data, the accuracy of this prediction for training set is 0.8439597315436241 and the accuracy of this prediction for test set is 0.8542713567839196.

***

### Question 3###
The rest of features in this dataset barring a few are categorical. No ML method accepts categorical features, so transform year, eyecolor, exercise into a set of binary features, one feature per unique original feature value, and mark the binary feature as ‘1’ if the feature value matches the original value and ‘0’ otherwise. Using only these binary variable transformed features, train and predict the class of the test set. What was your accuracy using Neural Network with a single 10 node hidden layer? During training, use a maximum number of iterations of 50.

In [11]:
# only get three features
df_train_bin = df_train[['year', 'eyecolor', 'exercise']]
df_test_bin = df_test[['year', 'eyecolor', 'exercise']]

#get dummy variable and convert to array
X_train_dummy = np.array(pd.get_dummies(df_train_bin))
y_train = df_train['gender']
X_test_dummy = np.array(pd.get_dummies(df_test_bin))
y_test = df_test['gender']

#label coding for target
le = LabelEncoder()
le.fit(['female', 'male'])
y_train = le.transform(y_train).reshape(-1, 1)
y_test = le.transform(y_test).reshape(-1, 1)

#keras training model
model = Sequential()
model.add(Dense(units=10, activation='sigmoid'))
model.add(Dense(units=1, activation='sigmoid'))
model.compile(loss='binary_crossentropy', optimizer='sgd', metrics=['accuracy'])
model.fit(X_train_dummy, y_train, epochs=50) #iter = 50

print('Accuracy on training---')
y_pred_scaled_train=model.predict(X_train_dummy)
y_pred_scaled_train=(y_pred_scaled_train>.5)*1
print(accuracy_score(y_train, y_pred_scaled_train))

print('')
print('Accuracy on test---')
y_pred_scaled_test=model.predict(X_test_dummy)
y_pred_scaled_test=(y_pred_scaled_test>.5)*1
print(accuracy_score(y_test, y_pred_scaled_test))


Epoch 1/50
38/38 [==============================] - 1s 886us/step - loss: 0.7376 - accuracy: 0.4622
Epoch 2/50
38/38 [==============================] - 0s 774us/step - loss: 0.7121 - accuracy: 0.4866
Epoch 3/50
38/38 [==============================] - 0s 808us/step - loss: 0.6991 - accuracy: 0.5159
Epoch 4/50
38/38 [==============================] - 0s 889us/step - loss: 0.6933 - accuracy: 0.5159
Epoch 5/50
38/38 [==============================] - 0s 808us/step - loss: 0.6905 - accuracy: 0.5369
Epoch 6/50
38/38 [==============================] - 0s 813us/step - loss: 0.6893 - accuracy: 0.5386
Epoch 7/50
38/38 [==============================] - 0s 805us/step - loss: 0.6886 - accuracy: 0.5369
Epoch 8/50
38/38 [==============================] - 0s 923us/step - loss: 0.6884 - accuracy: 0.5369
Epoch 9/50
38/38 [==============================] - 0s 1ms/step - loss: 0.6883 - accuracy: 0.5403
Epoch 10/50
38/38 [==============================] - 0s 943us/step - loss: 0.6882 - accuracy: 0.5411
E

###ANSWER: The accuracy of this prediction for training set is 0.5411073825503355 and the accuracy of this prediction for test set is 0.5402010050251256.




***
### Question 4###
Using a NN, report the accuracy on  the test set of a model that trained only on the height and the eye color features of instances in the training set.

**Question 4.a**  
What is the accuracy on the test set using the original height values (no pre-processing) and eye color as a one-hot?

In [12]:
# get the height feature
df_train_height = df_train[['height']]
df_test_height = df_test[['height']]

#get dummy variable and convert to array
X_train_dummy = pd.get_dummies(df_train[['eyecolor']])
y_train = df_train['gender']
X_test_dummy = pd.get_dummies(df_test[['eyecolor']])
y_test = df_test['gender']

X_train_q4 = X_train_dummy.join(df_train_height)
X_test_q4 = X_test_dummy.join(df_test_height)

#label coding for target
le = LabelEncoder()
le.fit(['female', 'male'])
y_train = le.transform(y_train).reshape(-1, 1)
y_test = le.transform(y_test).reshape(-1, 1)

#keras training model
model = Sequential()
model.add(Dense(units=10, activation='sigmoid'))
model.add(Dense(units=1, activation='sigmoid'))
model.compile(loss='binary_crossentropy', optimizer='sgd', metrics=['accuracy'])
model.fit(X_train_q4, y_train, epochs=50) #iter = 50

print('Accuracy on training---')
y_pred_scaled_train=model.predict(X_train_q4)
y_pred_scaled_train=(y_pred_scaled_train>.5)*1
print(accuracy_score(y_train, y_pred_scaled_train))

print('')
print('Accuracy on test---')
y_pred_scaled_test=model.predict(X_test_q4)
y_pred_scaled_test=(y_pred_scaled_test>.5)*1
print(accuracy_score(y_test, y_pred_scaled_test))

Epoch 1/50
38/38 [==============================] - 0s 980us/step - loss: 0.6956 - accuracy: 0.5428
Epoch 2/50
38/38 [==============================] - 0s 861us/step - loss: 0.6882 - accuracy: 0.5428
Epoch 3/50
38/38 [==============================] - 0s 867us/step - loss: 0.6871 - accuracy: 0.5428
Epoch 4/50
38/38 [==============================] - 0s 884us/step - loss: 0.6868 - accuracy: 0.5428
Epoch 5/50
38/38 [==============================] - 0s 1ms/step - loss: 0.6871 - accuracy: 0.5428
Epoch 6/50
38/38 [==============================] - 0s 915us/step - loss: 0.6870 - accuracy: 0.5428
Epoch 7/50
38/38 [==============================] - 0s 1ms/step - loss: 0.6871 - accuracy: 0.5428
Epoch 8/50
38/38 [==============================] - 0s 892us/step - loss: 0.6870 - accuracy: 0.5428
Epoch 9/50
38/38 [==============================] - 0s 862us/step - loss: 0.6874 - accuracy: 0.5428
Epoch 10/50
38/38 [==============================] - 0s 864us/step - loss: 0.6864 - accuracy: 0.5428
Epo

###ANSWER: 
Original height values (no pre-processing) and eye color as a one-hot:

The accuracy of this prediction for training set is 0.5427852348993288 and the accuracy of this prediction for test set is 0.5226130653266332.

**Question 4.b**  
What is the accuracy on the test set using the log of height values (applied to both training and testing sets) and eye color as a one-hot?

In [13]:
# apply log to the height
df_train_temp = df_train.copy()
df_train_temp['height'] = df_train_temp['height'].apply('log')
df_test_temp = df_test.copy()
df_test_temp['height'] = df_test_temp['height'].apply('log')

df_train_height = df_train_temp[['height']]
df_test_height = df_test_temp[['height']]

#get dummy variable and convert to array
X_train_dummy = pd.get_dummies(df_train[['eyecolor']])
y_train = df_train['gender']
X_test_dummy = pd.get_dummies(df_test[['eyecolor']])
y_test = df_test['gender']

X_train_q4b = X_train_dummy.join(df_train_height)
X_test_q4b = X_test_dummy.join(df_test_height)

#label coding for target
le = LabelEncoder()
le.fit(['female', 'male'])
y_train = le.transform(y_train).reshape(-1, 1)
y_test = le.transform(y_test).reshape(-1, 1)

#keras training model
model = Sequential()
model.add(Dense(units=10, activation='sigmoid'))
model.add(Dense(units=1, activation='sigmoid'))
model.compile(loss='binary_crossentropy', optimizer='sgd', metrics=['accuracy'])
model.fit(X_train_q4b, y_train, epochs=50) #iter = 50

print('Accuracy on training---')
y_pred_scaled_train=model.predict(X_train_q4b)
y_pred_scaled_train=(y_pred_scaled_train>.5)*1
print(accuracy_score(y_train, y_pred_scaled_train))

print('')
print('Accuracy on test---')
y_pred_scaled_test=model.predict(X_test_q4b)
y_pred_scaled_test=(y_pred_scaled_test>.5)*1
print(accuracy_score(y_test, y_pred_scaled_test))

Epoch 1/50
38/38 [==============================] - 0s 784us/step - loss: 0.7713 - accuracy: 0.5428
Epoch 2/50
38/38 [==============================] - 0s 849us/step - loss: 0.7218 - accuracy: 0.5428
Epoch 3/50
38/38 [==============================] - 0s 824us/step - loss: 0.7038 - accuracy: 0.5428
Epoch 4/50
38/38 [==============================] - 0s 958us/step - loss: 0.6975 - accuracy: 0.5428
Epoch 5/50
38/38 [==============================] - 0s 890us/step - loss: 0.6949 - accuracy: 0.5428
Epoch 6/50
38/38 [==============================] - 0s 1ms/step - loss: 0.6942 - accuracy: 0.5428
Epoch 7/50
38/38 [==============================] - 0s 855us/step - loss: 0.6938 - accuracy: 0.5428
Epoch 8/50
38/38 [==============================] - 0s 852us/step - loss: 0.6936 - accuracy: 0.5428
Epoch 9/50
38/38 [==============================] - 0s 792us/step - loss: 0.6937 - accuracy: 0.5428
Epoch 10/50
38/38 [==============================] - 0s 839us/step - loss: 0.6936 - accuracy: 0.5428
E

###ANSWER: 
The log of height values and eye color as a one-hot: 

The accuracy of this prediction for training set is 0.5427852348993288 and the accuracy of this prediction for test set is 0.5226130653266332.

**Question 4.c**  
What is the accuracy on the test set using the Z-score of height values and eye color as a one-hot? 

Z-score is a normalization function. It is the value of a feature minus the average value for that feature (in the training set), divided by the standard deviation of that feature (in the training set). Remember that, whenever applying a function to a feature in the training set, it also has to be applied to that same feature in the test set.

In [14]:
# z score function
h_mean = df_train['height'].mean()
h_std = df_train['height'].std()
calculate_Zscore = lambda x: (x - h_mean) / h_std

# apply z score to the height
df_train_temp = df_train.copy()
df_train_temp['height'] = df_train_temp['height'].apply(calculate_Zscore)
df_test_temp = df_test.copy()
df_test_temp['height'] = df_test_temp['height'].apply(calculate_Zscore)

df_train_height = df_train_temp[['height']]
df_test_height = df_test_temp[['height']]

#get dummy variable and convert to array
X_train_dummy = pd.get_dummies(df_train[['eyecolor']])
y_train = df_train['gender']
X_test_dummy = pd.get_dummies(df_test[['eyecolor']])
y_test = df_test['gender']

X_train_q4c = X_train_dummy.join(df_train_height)
X_test_q4c = X_test_dummy.join(df_test_height)

#label coding for target
le = LabelEncoder()
le.fit(['female', 'male'])
y_train = le.transform(y_train).reshape(-1, 1)
y_test = le.transform(y_test).reshape(-1, 1)

#keras training model
model = Sequential()
model.add(Dense(units=10, activation='sigmoid'))
model.add(Dense(units=1, activation='sigmoid'))
model.compile(loss='binary_crossentropy', optimizer='sgd', metrics=['accuracy'])
model.fit(X_train_q4c, y_train, epochs=50) #iter = 50

print('Accuracy on training---')
y_pred_scaled_train=model.predict(X_train_q4c)
y_pred_scaled_train=(y_pred_scaled_train>.5)*1
print(accuracy_score(y_train, y_pred_scaled_train))

print('')
print('Accuracy on test---')
y_pred_scaled_test=model.predict(X_test_q4c)
y_pred_scaled_test=(y_pred_scaled_test>.5)*1
print(accuracy_score(y_test, y_pred_scaled_test))

Epoch 1/50
38/38 [==============================] - 0s 833us/step - loss: 0.6521 - accuracy: 0.6577
Epoch 2/50
38/38 [==============================] - 0s 845us/step - loss: 0.6401 - accuracy: 0.7651
Epoch 3/50
38/38 [==============================] - 0s 992us/step - loss: 0.6311 - accuracy: 0.8104
Epoch 4/50
38/38 [==============================] - 0s 942us/step - loss: 0.6237 - accuracy: 0.8163
Epoch 5/50
38/38 [==============================] - 0s 915us/step - loss: 0.6174 - accuracy: 0.8020
Epoch 6/50
38/38 [==============================] - 0s 829us/step - loss: 0.6117 - accuracy: 0.8037
Epoch 7/50
38/38 [==============================] - 0s 899us/step - loss: 0.6060 - accuracy: 0.7995
Epoch 8/50
38/38 [==============================] - 0s 840us/step - loss: 0.6005 - accuracy: 0.7987
Epoch 9/50
38/38 [==============================] - 0s 958us/step - loss: 0.5951 - accuracy: 0.7911
Epoch 10/50
38/38 [==============================] - 0s 817us/step - loss: 0.5898 - accuracy: 0.8054

###ANSWER: 
The Z-score of height values and eye color as a one-hot:

The accuracy of this prediction for training set is 0.8053691275167785 and the accuracy of this prediction for test set is 0.7989949748743719.

***
### Question 5 ###
Repeat question 4 for exercise hours + eye color

In [15]:
#same as 4a: no prepocessing for hour + eyecolor one hot
#exercise hour features
df_train_hour = df_train[['exercisehours']]
df_test_hour = df_test[['exercisehours']]

#get dummy variable and convert to array
X_train_dummy = pd.get_dummies(df_train[['eyecolor']])
y_train = df_train['gender']
X_test_dummy = pd.get_dummies(df_test[['eyecolor']])
y_test = df_test['gender']

X_train_q5 = X_train_dummy.join(df_train_hour)
X_test_q5 = X_test_dummy.join(df_test_hour)

#label coding for target
le = LabelEncoder()
le.fit(['female', 'male'])
y_train = le.transform(y_train).reshape(-1, 1)
y_test = le.transform(y_test).reshape(-1, 1)

#keras training model
model = Sequential()
model.add(Dense(units=10, activation='sigmoid'))
model.add(Dense(units=1, activation='sigmoid'))
model.compile(loss='binary_crossentropy', optimizer='sgd', metrics=['accuracy'])
model.fit(X_train_q5, y_train, epochs=50) #iter = 50

print('Accuracy on training---')
y_pred_scaled_train=model.predict(X_train_q5)
y_pred_scaled_train=(y_pred_scaled_train>.5)*1
print(accuracy_score(y_train, y_pred_scaled_train))

print('')
print('Accuracy on test---')
y_pred_scaled_test=model.predict(X_test_q5)
y_pred_scaled_test=(y_pred_scaled_test>.5)*1
print(accuracy_score(y_test, y_pred_scaled_test))

Epoch 1/50
38/38 [==============================] - 0s 835us/step - loss: 0.7702 - accuracy: 0.4690
Epoch 2/50
38/38 [==============================] - 0s 1ms/step - loss: 0.7208 - accuracy: 0.5008
Epoch 3/50
38/38 [==============================] - 0s 910us/step - loss: 0.7005 - accuracy: 0.5159
Epoch 4/50
38/38 [==============================] - 0s 834us/step - loss: 0.6915 - accuracy: 0.5294
Epoch 5/50
38/38 [==============================] - 0s 845us/step - loss: 0.6870 - accuracy: 0.5587
Epoch 6/50
38/38 [==============================] - 0s 826us/step - loss: 0.6851 - accuracy: 0.5680
Epoch 7/50
38/38 [==============================] - 0s 810us/step - loss: 0.6843 - accuracy: 0.5654
Epoch 8/50
38/38 [==============================] - 0s 871us/step - loss: 0.6840 - accuracy: 0.5780
Epoch 9/50
38/38 [==============================] - 0s 1ms/step - loss: 0.6838 - accuracy: 0.5738
Epoch 10/50
38/38 [==============================] - 0s 937us/step - loss: 0.6836 - accuracy: 0.5763
Epo

Original exercisehours values (no pre-processing) and eye color as a one-hot:

The accuracy of this prediction for training set is 0.5780201342281879 and the accuracy of this prediction for test set is 0.5527638190954773.

In [16]:
df_train[df_train['exercisehours']  == 0]

,Unnamed: 0,gender,age,year,eyecolor,height,miles,brothers,sisters,computertime,exercise,exercisehours,musiccds,playgames,watchtv,gender_binary
0,577,male,20,third,hazel,72.0,180.0,0,0,5.0,No,0.0,100.0,10.0,10.0,0
5,878,female,27,third,hazel,67.0,0.0,2,1,2.0,No,0.0,108.0,1.0,4.0,1
6,1689,female,20,third,blue,64.0,50.0,1,0,10.0,No,0.0,20.0,0.0,15.0,1
8,1857,male,21,third,hazel,72.0,200.0,0,1,20.0,No,0.0,1000.0,1.0,4.0,0
12,809,female,20,second,brown,62.0,220.0,1,0,10.0,No,0.0,10.0,0.0,3.0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1175,587,male,20,third,blue,72.0,180.0,1,0,6.0,No,0.0,100.0,2.0,6.0,0
1176,94,female,21,third,brown,63.0,7000.0,0,1,10.0,No,0.0,300.0,0.0,4.0,1
1179,1834,female,20,second,green,71.0,120.0,1,2,25.0,No,0.0,30.0,0.0,4.0,1
1182,1953,female,21,third,green,68.0,30.0,1,0,7.0,No,0.0,20.0,0.0,10.0,1


In [17]:
df_train.shape

(1192, 16)

In [18]:
#same as 4b: log + one hot

#drop all the 0 value before converting to log base
df_train_temp = df_train.copy()
df_train_temp = df_train_temp[df_train_temp['exercisehours'] != 0].dropna()
df_test_temp = df_test.copy()
df_test_temp = df_test_temp[df_test_temp['exercisehours'] != 0].dropna()

# apply log to the exercisehours
df_train_temp['exercisehours'] = df_train_temp['exercisehours'].apply('log')
df_test_temp['exercisehours'] = df_test_temp['exercisehours'].apply('log')

df_train_exe = df_train_temp[['exercisehours']]
df_test_exe = df_test_temp[['exercisehours']]

#get dummy variable and convert to array
X_train_dummy = pd.get_dummies(df_train[['eyecolor']])
y_train = df_train['gender']
X_test_dummy = pd.get_dummies(df_test[['eyecolor']])
y_test = df_test['gender']

X_train_q5b = X_train_dummy.join(df_train_exe)
X_test_q5b = X_test_dummy.join(df_test_exe)

#label coding for target
le = LabelEncoder()
le.fit(['female', 'male'])
y_train = le.transform(y_train).reshape(-1, 1)
y_test = le.transform(y_test).reshape(-1, 1)

#keras training model
model = Sequential()
model.add(Dense(units=10, activation='sigmoid'))
model.add(Dense(units=1, activation='sigmoid'))
model.compile(loss='binary_crossentropy', optimizer='sgd', metrics=['accuracy'])
model.fit(X_train_q5b, y_train, epochs=50) #iter = 50

print('Accuracy on training---')
y_pred_scaled_train=model.predict(X_train_q5b)
y_pred_scaled_train=(y_pred_scaled_train>.5)*1
print(accuracy_score(y_train, y_pred_scaled_train))

print('')
print('Accuracy on test---')
y_pred_scaled_test=model.predict(X_test_q5b)
y_pred_scaled_test=(y_pred_scaled_test>.5)*1
print(accuracy_score(y_test, y_pred_scaled_test))

Epoch 1/50
38/38 [==============================] - 0s 862us/step - loss: nan - accuracy: 0.5428
Epoch 2/50
38/38 [==============================] - 0s 839us/step - loss: nan - accuracy: 0.5428
Epoch 3/50
38/38 [==============================] - 0s 1ms/step - loss: nan - accuracy: 0.5428
Epoch 4/50
38/38 [==============================] - 0s 990us/step - loss: nan - accuracy: 0.5428
Epoch 5/50
38/38 [==============================] - 0s 905us/step - loss: nan - accuracy: 0.5428
Epoch 6/50
38/38 [==============================] - 0s 905us/step - loss: nan - accuracy: 0.5428
Epoch 7/50
38/38 [==============================] - 0s 952us/step - loss: nan - accuracy: 0.5428
Epoch 8/50
38/38 [==============================] - 0s 892us/step - loss: nan - accuracy: 0.5428
Epoch 9/50
38/38 [==============================] - 0s 872us/step - loss: nan - accuracy: 0.5428
Epoch 10/50
38/38 [==============================] - 0s 1000us/step - loss: nan - accuracy: 0.5428
Epoch 11/50
38/38 [===========

The log of exercisehours values and eye color as a one-hot:

The accuracy of this prediction for training set is 0.5427852348993288 and the accuracy of this prediction for test set is 0.5226130653266332.

In [19]:
#same as 4c: z score + one hot
# z score function
e_mean = df_train['exercisehours'].mean()
e_std = df_train['exercisehours'].std()
calculate_Zscore = lambda x: (x - e_mean) / e_std

# apply z score to the exercisehours
df_train_temp = df_train.copy()
df_train_temp['exercisehours'] = df_train_temp['exercisehours'].apply(calculate_Zscore)
df_test_temp = df_test.copy()
df_test_temp['exercisehours'] = df_test_temp['exercisehours'].apply(calculate_Zscore)

df_train_exe = df_train_temp[['exercisehours']]
df_test_exe = df_test_temp[['exercisehours']]

#get dummy variable and convert to array
X_train_dummy = pd.get_dummies(df_train[['eyecolor']])
y_train = df_train['gender']
X_test_dummy = pd.get_dummies(df_test[['eyecolor']])
y_test = df_test['gender']

X_train_q5c = X_train_dummy.join(df_train_exe)
X_test_q5c = X_test_dummy.join(df_test_exe)

#label coding for target
le = LabelEncoder()
le.fit(['female', 'male'])
y_train = le.transform(y_train).reshape(-1, 1)
y_test = le.transform(y_test).reshape(-1, 1)

#keras training model
model = Sequential()
model.add(Dense(units=10, activation='sigmoid'))
model.add(Dense(units=1, activation='sigmoid'))
model.compile(loss='binary_crossentropy', optimizer='sgd', metrics=['accuracy'])
model.fit(X_train_q5c, y_train, epochs=50) #iter = 50

print('Accuracy on training---')
y_pred_scaled_train=model.predict(X_train_q5c)
y_pred_scaled_train=(y_pred_scaled_train>.5)*1
print(accuracy_score(y_train, y_pred_scaled_train))

print('')
print('Accuracy on test---')
y_pred_scaled_test=model.predict(X_test_q5c)
y_pred_scaled_test=(y_pred_scaled_test>.5)*1
print(accuracy_score(y_test, y_pred_scaled_test))

Epoch 1/50
38/38 [==============================] - 0s 921us/step - loss: 0.7337 - accuracy: 0.4564
Epoch 2/50
38/38 [==============================] - 0s 929us/step - loss: 0.7167 - accuracy: 0.4547
Epoch 3/50
38/38 [==============================] - 0s 918us/step - loss: 0.7073 - accuracy: 0.4488
Epoch 4/50
38/38 [==============================] - 0s 896us/step - loss: 0.7024 - accuracy: 0.4757
Epoch 5/50
38/38 [==============================] - 0s 878us/step - loss: 0.7000 - accuracy: 0.4849
Epoch 6/50
38/38 [==============================] - 0s 868us/step - loss: 0.6986 - accuracy: 0.4933
Epoch 7/50
38/38 [==============================] - 0s 1ms/step - loss: 0.6977 - accuracy: 0.5042
Epoch 8/50
38/38 [==============================] - 0s 955us/step - loss: 0.6971 - accuracy: 0.5076
Epoch 9/50
38/38 [==============================] - 0s 845us/step - loss: 0.6966 - accuracy: 0.5252
Epoch 10/50
38/38 [==============================] - 0s 885us/step - loss: 0.6962 - accuracy: 0.5252
E

The Z-score of exercisehours values and eye color as a one-hot:

The accuracy of this prediction for training set is 0.5528523489932886 and the accuracy of this prediction for test set is 0.550251256281407.

***
### Question 6###
Combine the features from question 3, 4, and 5 (year, eyecolor, exercise, height, exercise hours). For numeric features use the best normalization method from questions 4 and 5.

**Question 6.a**  
What was the NN accuracy on the test set using the single 10 node hidden layer?

In [44]:
#best normalization:
# q4 -- The Z-score of height values and eye color as a one-hot
# q5 -- The Z-score of exercise hours values and eye color as a one-hot

# q3:
df_train_bin = df_train[['year', 'eyecolor', 'exercise']]
df_test_bin = df_test[['year', 'eyecolor', 'exercise']]
X_train_dummy = pd.get_dummies(df_train_bin)
X_test_dummy = pd.get_dummies(df_test_bin)

# q4: z score function
h_mean = df_train['height'].mean()
h_std = df_train['height'].std()
calculate_Zscore = lambda x: (x - h_mean) / h_std

# apply z score to the height
df_train_temp = df_train.copy()
df_train_temp['height'] = df_train_temp['height'].apply(calculate_Zscore)
df_test_temp = df_test.copy()
df_test_temp['height'] = df_test_temp['height'].apply(calculate_Zscore)

df_train_height = df_train_temp[['height']]
df_test_height = df_test_temp[['height']]

# q5: z score function
e_mean = df_train['exercisehours'].mean()
e_std = df_train['exercisehours'].std()
calculate_Zscore = lambda x: (x - e_mean) / e_std

# apply z score to the exercisehours
df_train_temp = df_train.copy()
df_train_temp['exercisehours'] = df_train_temp['exercisehours'].apply(calculate_Zscore)
df_test_temp = df_test.copy()
df_test_temp['exercisehours'] = df_test_temp['exercisehours'].apply(calculate_Zscore)

df_train_exe = df_train_temp[['exercisehours']]
df_test_exe = df_test_temp[['exercisehours']]

# combine table
df_train_q6 = X_train_dummy.join(df_train_height)
df_test_q6 = X_test_dummy.join(df_test_height)
df_train_q6 = df_train_q6.join(df_train_exe)
df_test_q6 = df_test_q6.join(df_test_exe)

#target
y_train = df_train['gender']
y_test = df_test['gender']

#label coding for target
le = LabelEncoder()
le.fit(['female', 'male'])
y_train = le.transform(y_train).reshape(-1, 1)
y_test = le.transform(y_test).reshape(-1, 1)

# keras training model
model = Sequential()
model.add(Dense(units=10, activation='sigmoid'))
model.add(Dense(units=1, activation='sigmoid'))
model.compile(loss='binary_crossentropy', optimizer='sgd', metrics=['accuracy'])
model.fit(df_train_q6, y_train, epochs=50) #iter = 50

print('Accuracy on training---')
y_pred_scaled_train=model.predict(df_train_q6)
y_pred_scaled_train=(y_pred_scaled_train>.5)*1
print(accuracy_score(y_train, y_pred_scaled_train))

print('')
print('Accuracy on test---')
y_pred_scaled_test=model.predict(df_test_q6)
y_pred_scaled_test=(y_pred_scaled_test>.5)*1
print(accuracy_score(y_test, y_pred_scaled_test))

Epoch 1/50
38/38 [==============================] - 0s 798us/step - loss: 0.7127 - accuracy: 0.3901
Epoch 2/50
38/38 [==============================] - 0s 979us/step - loss: 0.7058 - accuracy: 0.4891
Epoch 3/50
38/38 [==============================] - 0s 1ms/step - loss: 0.7004 - accuracy: 0.5201
Epoch 4/50
38/38 [==============================] - 0s 885us/step - loss: 0.6955 - accuracy: 0.5361
Epoch 5/50
38/38 [==============================] - 0s 1ms/step - loss: 0.6910 - accuracy: 0.5428
Epoch 6/50
38/38 [==============================] - 0s 837us/step - loss: 0.6867 - accuracy: 0.5445
Epoch 7/50
38/38 [==============================] - 0s 830us/step - loss: 0.6825 - accuracy: 0.5453
Epoch 8/50
38/38 [==============================] - 0s 901us/step - loss: 0.6783 - accuracy: 0.5453
Epoch 9/50
38/38 [==============================] - 0s 959us/step - loss: 0.6743 - accuracy: 0.5487
Epoch 10/50
38/38 [==============================] - 0s 864us/step - loss: 0.6702 - accuracy: 0.5495
Epo

###ANSWER: 
The accuracy of this prediction for training set is 0.8238255033557047 and the accuracy of this prediction for test set is 0.821608040201005.

***
### Question 7- Bonus (10%)###
Can you improve your test set prediction accuracy by 5% or more?  

See how close to that milestone of improvement you can get by modifying the tuning parameters of  Neural Networks (the number of hidden layers, number of hidden nodes in each layer, the learning rate aka mu). A great guide to tuning parameters is explained in this guide: http://www.csie.ntu.edu.tw/~cjlin/papers/guide/guide.pdf. 

While the guide is specific to SVM and in particular the C and gamma parameters of the RBF kernel, the method applies to generally to any ML technique with tuning parameters.

Please also write a paragraph in a markdown cell below with an explanation of your approach and evaluation metrics.


In [116]:
from keras.layers import Dropout, BatchNormalization
from tensorflow.keras.optimizers import SGD
from keras.constraints import maxnorm

#best normalization:
# q4 -- The Z-score of height values and eye color as a one-hot
# q5 -- The Z-score of exercise hours values and eye color as a one-hot

# q3:
df_train_bin = df_train[['year', 'eyecolor', 'exercise']]
df_test_bin = df_test[['year', 'eyecolor', 'exercise']]
X_train_dummy = pd.get_dummies(df_train_bin)
X_test_dummy = pd.get_dummies(df_test_bin)

# q4: z score function
h_mean = df_train['height'].mean()
h_std = df_train['height'].std()
calculate_Zscore = lambda x: (x - h_mean) / h_std

# apply z score to the height
df_train_temp = df_train.copy()
df_train_temp['height'] = df_train_temp['height'].apply(calculate_Zscore)
df_test_temp = df_test.copy()
df_test_temp['height'] = df_test_temp['height'].apply(calculate_Zscore)

df_train_height = df_train_temp[['height']]
df_test_height = df_test_temp[['height']]

# q5: z score function
e_mean = df_train['exercisehours'].mean()
e_std = df_train['exercisehours'].std()
calculate_Zscore = lambda x: (x - e_mean) / e_std

# apply z score to the exercisehours
df_train_temp = df_train.copy()
df_train_temp['exercisehours'] = df_train_temp['exercisehours'].apply(calculate_Zscore)
df_test_temp = df_test.copy()
df_test_temp['exercisehours'] = df_test_temp['exercisehours'].apply(calculate_Zscore)

df_train_exe = df_train_temp[['exercisehours']]
df_test_exe = df_test_temp[['exercisehours']]

# combine table
df_train_q6 = X_train_dummy.join(df_train_height)
df_test_q6 = X_test_dummy.join(df_test_height)
df_train_q6 = df_train_q6.join(df_train_exe)
df_test_q6 = df_test_q6.join(df_test_exe)

#target
y_train = df_train['gender']
y_test = df_test['gender']

#label coding for target
le = LabelEncoder()
le.fit(['female', 'male'])
y_train = le.transform(y_train).reshape(-1, 1)
y_test = le.transform(y_test).reshape(-1, 1)

#keras training model
model = Sequential()
# model.add(Dropout(0.2, input_shape=(14,)))
model.add(Dense(units=200, activation='sigmoid'))
model.add(Dense(units=1, activation='sigmoid'))
model.compile(loss='binary_crossentropy', optimizer='sgd', metrics=['accuracy']) #sgd, adam
model.fit(df_train_q6, y_train, epochs=300) #iter = 300 batch_size=512,

print('Accuracy on training---')
y_pred_scaled_train=model.predict(df_train_q6)
y_pred_scaled_train=(y_pred_scaled_train>.5)*1
print(accuracy_score(y_train, y_pred_scaled_train))

print('')
print('Accuracy on test---')
y_pred_scaled_test=model.predict(df_test_q6)
y_pred_scaled_test=(y_pred_scaled_test>.5)*1
print(accuracy_score(y_test, y_pred_scaled_test))

Epoch 1/300
38/38 [==============================] - 0s 906us/step - loss: 0.6943 - accuracy: 0.5394
Epoch 2/300
38/38 [==============================] - 0s 877us/step - loss: 0.6834 - accuracy: 0.5529
Epoch 3/300
38/38 [==============================] - 0s 1ms/step - loss: 0.6729 - accuracy: 0.6040
Epoch 4/300
38/38 [==============================] - 0s 1ms/step - loss: 0.6646 - accuracy: 0.5872
Epoch 5/300
38/38 [==============================] - 0s 852us/step - loss: 0.6549 - accuracy: 0.6300
Epoch 6/300
38/38 [==============================] - 0s 984us/step - loss: 0.6461 - accuracy: 0.6300
Epoch 7/300
38/38 [==============================] - 0s 922us/step - loss: 0.6374 - accuracy: 0.7206
Epoch 8/300
38/38 [==============================] - 0s 908us/step - loss: 0.6285 - accuracy: 0.7206
Epoch 9/300
38/38 [==============================] - 0s 926us/step - loss: 0.6209 - accuracy: 0.7819
Epoch 10/300
38/38 [==============================] - 0s 1ms/step - loss: 0.6130 - accuracy: 0.

###ANSWER: 
The test accuracy improved 4% from 0.821608040201005 to 0.8618090452261307. 

1. I set the epochs to 300, so the entire dataset will be passed forward and backward through the neural network 300 times.

2. Then I also change the dense units to 200, dimensionality of the output space. It is the unit parameter itself that plays a major role in the size of the weight matrix along with the bias vector.